In [1]:
import h2o
h2o.init()
from h2o.estimators.word2vec import H2OWord2vecEstimator

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "10.0.2" 2018-07-17; Java(TM) SE Runtime Environment 18.3 (build 10.0.2+13); Java HotSpot(TM) 64-Bit Server VM 18.3 (build 10.0.2+13, mixed mode)
  Starting server from /Users/karthik/venv/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/9b/rjdvjn8s3rgcbsvrxmt9rzt00000gp/T/tmpe_cl95aq
  JVM stdout: /var/folders/9b/rjdvjn8s3rgcbsvrxmt9rzt00000gp/T/tmpe_cl95aq/h2o_karthik_started_from_python.out
  JVM stderr: /var/folders/9b/rjdvjn8s3rgcbsvrxmt9rzt00000gp/T/tmpe_cl95aq/h2o_karthik_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.7
H2O_cluster_version_age:,2 months and 4 days
H2O_cluster_name:,H2O_from_python_karthik_cvgod3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [2]:
news = h2o.upload_file("news.csv", destination_frame = "tamiltext",
                             col_names = ["news_in_tamil"], col_types = ["string"], header = 1, sep="\n")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
news

news_in_tamil
﻿ஈராக்கில் 43 ஆண்டுகள் கழித்து அழகிப்போட்டி
இந்திய அளவில் ட்ரெண்ட் ஆன அஜித்தின் ‘தள்லே தில்லாலே’
சொந்த செலவில் வாகன காப்பீடு எடுத்து கொடுத்த காவல் ஆய்வாளர்
பிறந்தநாளில் பட்ஜெட் தாக்கல் செய்தவர்
“நானா? பாஜகவில் இணைந்துவிட்டேனா” - வரலட்சுமி விளக்கம்
இஸ்லாமிய மதபோதகர் ஜாகிர் நாயக் மீது 58 பக்க குற்றப்பத்திரிக்கை தாக்கல்
மேயாதமான் இசையமைப்பாளர்களுக்கு மிகப்பெரிய வாழ்த்து: தனுஷ் ட்வீட்
அடுத்த 50 ஆண்டுக்கு பாஜக ஆட்சி - அமித் ஷா
"தொடரும் பஸ் கட்டண உயர்வு: தினசரி, மாதாந்திர பாஸ்-ம் காலி!"
செம்மரம் வெட்டச் சென்றதாக திருப்பதியில் தமிழர்கள் கைது


In [6]:
# Optional Stopwords. Currently its empty
STOP_WORDS = []

In [7]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize(" ")
    print(tokenized)
    tokenized_lower = tokenized.tolower() # Do you need it ? Not sure
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    #tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [8]:
print("Break tamil_text into sequence of words")
words = tokenize(news["news_in_tamil"])

Break tamil_text into sequence of words


C1
﻿ஈராக்கில்
43
ஆண்டுகள்
கழித்து
அழகிப்போட்டி
""
இந்திய
அளவில்
ட்ரெண்ட்
ஆன


In [9]:
# Build Word2Vec Model

w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
w2v_model.train(training_frame=words)

word2vec Model Build progress: |█████████████████████████████████████████████████| (done) 100%
Model Details
H2OWord2vecEstimator :  Word2Vec
Model Key:  Word2Vec_model_python_1645909001674_1

No model summary for this model


In [10]:
# Check Synonym for a term
dict = w2v_model.find_synonyms("இந்திய", count = 5)

In [11]:
for key, value in dict.items():
    print(key, value)

இங்கிலாந்து 0.7148316502571106
வெற்றி! 0.7089929580688477
சதம்: 0.6825829148292542
ஆஸி. 0.6724561452865601
போட்டி: 0.6568611264228821


In [60]:
word_vecs = w2v_model.transform(words, aggregate_method = "AV")

In [61]:
word_vecs

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
-0.141272,0.0210371,-0.150056,-0.161734,0.226709,0.133739,0.04881,0.245209,-0.321804,0.278549,-0.12475,0.121165,-0.315182,-0.0419019,-0.0548095,0.15064,-0.209577,-0.0322639,-0.0480403,0.112338,-0.300634,0.0287526,-0.1759,0.0301511,-0.252091,-0.0520743,0.205384,0.0741519,-0.241981,0.165226,0.0299898,0.084553,-0.150568,-0.382158,-0.0681222,0.166749,-0.142141,-0.181063,0.251345,0.0368071,-0.109512,-0.171217,0.221356,-0.0199538,-0.16568,0.222681,-0.267069,0.374294,0.243968,0.12201,-0.350507,0.232151,0.467265,0.114159,-0.036064,-0.0777867,-0.0739533,-0.0298974,0.275089,0.41218,-0.217924,-0.146125,0.170981,-0.0266488,0.0455067,0.127442,0.0698578,-0.0654624,-8.48727e-05,0.18605,-0.0418112,-0.165813,-0.181578,-0.0524305,0.126765,0.000186053,0.101987,0.011538,0.110111,0.0645214,0.0500511,0.314926,-0.600724,-0.0214577,0.0214799,-0.106994,-0.0310496,0.275871,0.161633,-0.0450546,0.0747054,-0.227428,-0.0319143,-0.0323625,0.110809,-0.395503,-0.025054,0.0744167,0.312533,-0.0559482
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0.357215,-0.472243,-0.340513,-0.129482,0.146913,-0.179203,0.0641688,-0.272363,0.169561,0.326539,-0.222898,0.140319,-0.198716,0.334065,0.0810766,0.29859,0.334071,0.205581,0.450729,0.00784545,-0.0205487,-0.271439,0.452036,-0.0209

In [64]:
model_path = h2o.save_model(model = w2v_model, path= "" ,force=True)